In [1]:
import os
import numpy as np
import pandas as pd
from brainrender import Scene, Animation, settings
from brainrender.actors import Points
from brainrender._colors import get_random_colors
from brainrender._video import Video
from vedo import embedWindow, Plotter, show
from brainrender.video import VideoMaker
import brainrender as br
from myterial import amber, orange, salmon
from scipy.spatial.distance import pdist

In [2]:
directory = r'C:\Users\shirahad\Documents\research\electrophysiology\analyze_electrophysiology_data\brain_render\files'

In [3]:
probes_number = 0
clusters_sets_number = 1
scene_title = 'clusters mapping'
brain_regions = ['ILA1', 'ILA2/3', 'ILA5']
#brain_regions = ['ACAd1', 'ACAd2/3', 'ACAd5', 'ILA1', 'ILA2/3', 'ILA5', 'PL1', 'PL2/3', 'PL5'] #['DG', 'MD', 'LH']

In [4]:
clusters_sets_files = ['clustering_data.csv']
probes_files = []

In [ ]:
probes_files = ['probe_coordinates.csv'] #['probe_coordinates_1.csv', 'probe_coordinates_2.csv']
clusters_sets_files = ['clusters_coordinates.csv'] #['clusters_coordinates_1.csv', 'clusters_coordinates_2.csv']

In [5]:
def add_probes_to_scene(scene, probes_number, probes_files, directory):
    if probes_files is None:
        return scene
    probes_colors = get_random_colors(n_colors=probes_number)
    for probe_file, probe_color in zip(probes_files, probes_colors):
        probe_file_full_path = os.path.join(directory, probe_file)   
        probe_coordinates = np.loadtxt(open(probe_file_full_path, 'r'), delimiter=',', skiprows=1, usecols=(3, 2, 1))   
        brain_regions = pd.read_csv(probe_file_full_path, header=0, index_col=0, usecols=['id', 'region_acronym']).drop_duplicates() 
        # Add to scene
        scene.add(Points(probe_coordinates, name='probe', colors=probe_color))
        # Add relevant brain regions
        scene = add_brain_regions_to_scene(scene, brain_regions['region_acronym'].tolist())
    return scene

In [48]:
def add_clustering_sets_to_scene(scene, clusters_sets_files, directory):
    for clusters_file in clusters_sets_files:
        clusters_file_full_path = os.path.join(directory, clusters_file)
        clusters_df = pd.read_csv(clusters_file_full_path, header=0, index_col=0, usecols=['id', 'x', 'y', 'z', 'cluster']) 
        cluster_groups = clusters_df.groupby(['cluster'])
        colors = get_random_colors(n_colors=len(cluster_groups))
        # Add to scene
        for (name, group), color in zip(cluster_groups, colors):
            clusters_coordinates = group[['z', 'y', 'x']].to_numpy() # AP, DV, ML
            scene.add(Points(clusters_coordinates, name='cluster_{}'.format(name), colors=color))
    return scene

In [8]:
def add_clusters_to_scene(scene, clusters_sets_files, directory):
    for clusters_file in clusters_sets_files:
        clusters_file_full_path = os.path.join(directory, clusters_file)
        clusters_coordinates = np.loadtxt(open(clusters_file_full_path, 'r'), delimiter=',', skiprows=1, usecols=(3, 2, 1))
        colors = get_random_colors(n_colors=len(clusters_coordinates))
        scene.add(Points(clusters_coordinates, name='clusters', colors=colors)) # add to scene
    return scene

In [9]:
def add_brain_regions_to_scene(scene, brain_regions):
    colors = get_random_colors(n_colors=len(brain_regions))
    for region, color in zip(brain_regions, colors):
        scene.add_brain_region(region, alpha=0.2, color=color)
    return scene

In [49]:
scene = Scene(title=scene_title)

clustering = list(filter(lambda x: 'clusters_coordinates' not in x, clusters_sets_files))
cells = list(filter(lambda x: 'clusters_coordinates' in x, clusters_sets_files))

if probes_files is not None:
    scene = add_probes_to_scene(scene, probes_number, probes_files, directory)
if not 0 == len(clustering):
    scene = add_clustering_sets_to_scene(scene, clusters_sets_files, directory)
if not 0 == len(cells):
    scene = add_clusters_to_scene(scene, clusters_sets_files, directory)
scene = add_brain_regions_to_scene(scene, brain_regions)

scene.content
scene.render()

embedWindow('k3d')

scene.jupyter = True

plt = Plotter()
plt.show(*scene.renderables)

╭───────────────────────────────────────────────────────────╮
│ │
│ Scene actors │
│ ──────────── │
│ │
│ - root ( type: brain region ) | is transformed: False │
│ - Actor ( type: None ) | is transformed: False │
│ - cluster_1 ( type: Points ) | is transformed: False │
│ - cluster_2 ( type: Points ) | is transformed: False │
│ - cluster_3 ( type: Points ) | is transformed: False │
│ - ILA1 ( type: brain region ) | is transformed: False │
│ - ILA2/ 3 ( type: brain region ) | is transformed: False │
│ - ILA5 ( type: brain region ) | is transformed: False │
│ │
╰───────────────────────────────────────────────────────────╯

Error in silhouette(): direction is [None]
 render the scene with show() or specify camera/direction
Error in silhouette(): direction is [None]
 render the scene with show() or specify camera/direction
Error in silhouette(): direction is [None]
 render the scene with show() or specify camera/direction
Error in silhouette(): direction is [None]
 render the scene with show() or specify camera/direction


Your scene is ready for rendering, use: `show ( scene.renderables ) `

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[6771.671281616228,…